In [1]:
import mediapipe as mp 
import cv2
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose
cap = cv2.VideoCapture(0)

with mp_pose.Pose(min_detection_confidence=0.5,min_tracking_confidence=0.5) as pose:

    while (cap.isOpened()):
        succes,image = cap.read()
        if not succes:
            print("Couldnt load webcam")
            break 
        image = cv2.cvtColor(cv2.flip(image, 1), cv2.COLOR_BGR2RGB)
        #image.flags.writeable = False
        #results = pose.process(image)
        #image.flags.writeable = True 
        #image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
        #print(results.pose_landmarks)
        #mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS)
        cv2.imshow('MediaPipe Pose', image)

        if cv2.waitKey(5) & 0xFF == 27:
            break
        cap.release()

In [2]:
"""
Bekijken van de base class methods
"""

class TestClass:
    def __init__(self,x,y):
        self.x = x 
        self.y = y
    def __call__(self):
        print("Using __call__ method")
    def __str__(self):
        return "x: %.2d, y: %.2d" % (self.x,self.y)
    def __len__(self):
        print("Using __len__ method")
        return 2
test = TestClass(10,16)
test()
print(str(test))
print(len(test))
print(test)

Using __call__ method
x: 10, y: 16
Using __len__ method
2
x: 10, y: 16


In [4]:
import torch 
import torchvision 
import torchvision.transforms as transforms 


transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

batch_size = 4

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=batch_size,
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=batch_size,
                                         shuffle=False, num_workers=2)

classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
import matplotlib.pyplot as plt
import numpy as np

# functions to show an image


def imshow(img):
    img = img / 2 + 0.5     # unnormalize
    npimg = img.numpy()
    plt.imshow(np.transpose(npimg, (1, 2, 0)))
    plt.show()


# get some random training images
dataiter = iter(trainloader)
images, labels = dataiter.next()

# show images
imshow(torchvision.utils.make_grid(images))
# print labels
print(' '.join('%5s' % classes[labels[j]] for j in range(batch_size)))




In [6]:
import torch.nn as nn 
import torch.nn.functional as F 
import torch.optim as optim 

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)
class Net(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 =nn.Conv2d(3,6,5)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 16, 5)
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = torch.flatten(x, 1) # flatten all dimensions except batch
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def backward(self):
        assert("Not implemented yet")
        

net = Net() 
criterion = nn.CrossEntropyLoss() 
optimizer = optim.SGD(net.parameters(),lr =0.001, momentum=0.9)


for epoch in range(3):
    running_loss = 0.
    for i,data in enumerate(trainloader,0):
        inputs,label =data 
        optimizer.zero_grad()
        outputs = net(inputs)
        loss = criterion(outputs,label)
        loss.backward()
        optimizer.step() 
        running_loss += loss.item() 
        if i % 2000 == 1999:
            print(" %d, %.5d loss: %.3f" % (epoch +1, i+1 ,running_loss/2000))
            running_loss= 0.0


print("Finished training")

.1,02000 loss: 2.196
.1,04000 loss: 1.900
.1,06000 loss: 1.716
.1,08000 loss: 1.608
.1,10000 loss: 1.517
.1,12000 loss: 1.495
.2,02000 loss: 1.429
.2,04000 loss: 1.395
.2,06000 loss: 1.386
.2,08000 loss: 1.357
.2,10000 loss: 1.295
.2,12000 loss: 1.330
.3,02000 loss: 1.214
.3,04000 loss: 1.229
.3,06000 loss: 1.228
.3,08000 loss: 1.230
.3,10000 loss: 1.218
.3,12000 loss: 1.185
Finished training


In [10]:
"""Creating predictions using the train network (net) from above """
dataiter = iter(testloader)
images,labels = dataiter.next() 
outputs = net(images)

_, predicted =torch.max(outputs,1)
print("Predicted: ", "".join('%5s' % classes[predicted[j]] for j in range(4)))

correct = 0 
total = 0 
with torch.no_grad():
    for data in testloader:
        images,labels = data 
        outputs = net(images)
        _, predicted = torch.max(outputs,1)
        total += labels.size(0)
        correct += (predicted ==labels).sum().item()
print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Predicted:   deer ship shipplane
Accuracy of the network on the 10000 test images: 58 %
